In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
import re
import string
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st
import faiss



In [2]:
url = '/clean_mail_movie.csv'

df = pd.read_csv(url)

In [4]:
dataset = df['concat2embedding'].tolist() # Это объединённый столбец
titles = df['movie_title'].tolist()
images = df['image_url'].tolist()
descr = df['description'].tolist()
links = df['page_url'].tolist()

In [5]:
def clean(text):
    text = text.lower()  # Нижний регистр
    # text = re.sub(r'\d+', ' ', text)  # Удаляем числа
    # text = text.translate(str.maketrans('', '', string.punctuation))  # Удаляем пунктуацию
    text = re.sub(r'\s+', ' ', text)  # Удаляем лишние пробелы
    text = text.strip()  # Удаляем начальные и конечные пробелы
    # text = re.sub(r'\b\w{1,2}\b', '', text)  # Удаляем слова длиной менее 3 символов
    # Дополнительные шаги, которые могут быть полезны в данном контексте:
    # text = re.sub(r'\b\w+\b', '', text)  # Удаляем отдельные слова (без чисел и знаков препинания)
    # text = ' '.join([word for word in text.split() if word not in stop_words])  # Удаляем стоп-слова
    return text


cleaned_text = []

for text in dataset:
    cleaned_text.append(clean(text))

In [6]:
# pip install transformers sentencepiece

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
# model.cuda()  # uncomment it if you have a GPU

In [7]:
# Дефолтная функция, шла в комплекте с моделью

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt', max_length=1024) # Модель сама создаёт пэддинги и маску.
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [8]:
# Векторизация отзывов
text_embeddings = np.array([embed_bert_cls(text, model, tokenizer) for text in cleaned_text])

In [9]:
# Создание FAISS индекса после определения text_embeddings
dimension = text_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(text_embeddings.astype('float32'))

In [10]:
from joblib import dump, load

# Сохранение эмбеддингов
dump(text_embeddings, 'mail_embeddings.joblib')

['mail_embeddings.joblib']

In [11]:
# Сохранение индекса
faiss.write_index(index, "mail_faiss_index.index")